In [1]:
import os

In [2]:
%pwd

'/Users/main/Desktop/LSTM_Forecast/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/main/Desktop/LSTM_Forecast'

In [5]:
from TimeSeriesForecast import logger
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy as dc
import torch
from TimeSeriesForecast.constants import *
from TimeSeriesForecast.utils.common import read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreparationConfig:
    root_dir: Path
    data_path: Path
    save_path: Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_preparation_config(self)->DataPreparationConfig:
        config= self.config.data_preparation

        create_directories([config.root_dir])

        data_preparation_config= DataPreparationConfig(
                                                        root_dir= config.root_dir,
                                                        data_path= config.data_path,
                                                        save_path= config.save_path
                                                        )
        return data_preparation_config

class DataPreparation:
    def __init__(self,config:DataPreparationConfig):
        self.config= config
        self.df= None
        self.X_train= None
        self.X_test= None
        self.y_test= None
        self.y_train= None
        self.y_test= None
    def extract_data_for_preparation(self):
        try:
            self.df= pd.read_csv(self.config.data_path)
            self.df= self.df.to_numpy()
            return self
        except Exception as e:
            raise Exception(f'Problem loading the data: {e}')
    def scaling_data(self):
        try:
            scaler= MinMaxScaler(feature_range=(-1,1))
            transformed= scaler.fit_transform(self.df)
            self.transformed= transformed
            return self
        except Exception as e:
            raise Exception(f'Problem Scaling the data: {e}')
    def transform_split_data(self):
        try:
            X=dc(np.flip(X,axis=1))
            X= self.transformed[:,1:]
            y= self.transformed[:,0]
            split_index= int(len(X)*0.80)
            self.X_train= X[:split_index]
            self.X_test= X[split_index:]
            self.y_train= y[:split_index]
            self.y_test= y[split_index:]
            logger.info('Transform and split done!')
            return self
        except Exception as e:
            raise Exception(f'Problem transforming and splitting the data: {e}')
    def reshaping_data(self):
        try:
            self.X_train= self.X_train.reshape((-1,10,1))
            self.X_test= self.X_test.reshape((-1,10,1))
            self.y_train= self.y_train.reshape((-1,1))
            self.y_test= self.y_test.reshape((-1,1))
            self.X_train= torch.tensor(self.X_train).float()
            self.X_test= torch.tensor(self.X_test).float()
            self.y_train= torch.tensor(self.y_train).float()
            self.y_test= torch.tensor(self.y_test).float()
            return self
        except Exception as e:
            raise Exception(f'Problem reshaping and creating tensors {e}')
    def saving_tensors(self):
        try:
            torch.save({'X_train':self.X_train,'X_test':self.X_test,
                        'y_train':self.y_train,'y_test':self.y_test},self.config.save_path)
            print(f'Tensors have succesfuly saved in: {self.config.save_path}')
            logger.info(f'Tensors succesfully saved in: {self.config.save_path}')
        except Exception as e:
            raise Exception(f'Problem saving the tensors: {e}')
        
try:
        config= ConfigurationManager()
        data_preparation_config= config.get_data_preparation_config()
        data_preparation= DataPreparation(config=data_preparation_config)
        data_preparation.extract_data_for_preparation()
        data_preparation.scaling_data()
        data_preparation.transform_split_data()
        data_preparation.reshaping_data()
        data_preparation.saving_tensors()
except Exception as e:
    raise e    
        

[2024-08-02 22:49:47,201:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-08-02 22:49:47,202:INFO:common:yaml file: params.yaml loaded successfully]
[2024-08-02 22:49:47,203:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-08-02 22:49:47,203:INFO:common:created directory at: artifacts]
[2024-08-02 22:49:47,204:INFO:common:created directory at: artifacts/data_preparation]
[2024-08-02 22:49:47,206:INFO:3293646955:Transform and split done!]
Tensors have succesfuly saved in: artifacts/data_preparation/tensors.pt
[2024-08-02 22:49:47,208:INFO:3293646955:Tensors succesfully saved in: artifacts/data_preparation/tensors.pt]
